In [ ]:
# To run in Google Colab, uncomment the next line
# !pip install -qq weaviate-client python-dotenv weaviate-agents datasets

# Query Agent

<a target="_blank" href="https://colab.research.google.com/github/saskinosie/dsd-building-ai-agents-with-vector-db/blob/main/3-bonus-weaviate_query_agent.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Here, we'll walk you through a comprehensive example showcasing the **Weaviate Query Agent** functionality.  
The Query Agent is an intelligent layer that sits on top of your Weaviate vector database, using generative AI to optimize natural language queries and automatically determine the best search strategy.

### How the Query Agent Works

The Query Agent intelligently handles complex queries by:
- **Query Optimization**: Uses generative AI to transform natural language into optimized vector database queries
- **Collection Selection**: Automatically determines which collection(s) to search based on your question
- **Smart Filtering**: Decides when filtering is needed to narrow down results
- **Aggregation Logic**: Determines if aggregation operations should be performed (counts, grouping, etc.)
- **Execution**: Runs the optimized queries against your Weaviate instance

### Dataset Overview

This notebook demonstrates the Query Agent using three diverse collections:
- **Books**: 10,000 books with titles, authors, descriptions, and genres
- **Brands**: 104 clothing brands with descriptions, ratings, and company information  
- **Ecommerce**: 448 fashion items with prices, categories, reviews, and brand associations

All datasets come pre-vectorized with **Snowflake Arctic Embed v2.0** embeddings via Weaviate's embedding service.

### Further Reading

- Check out the datasets on [HuggingFace](https://huggingface.co/datasets/weaviate/agents)
- Learn more about Query Agents in the [official documentation](https://docs.weaviate.io/agents/query)
- Explore the technical implementation in our [Query Agent tutorial](https://docs.weaviate.io/agents/query/tutorial-ecommerce)
- Understand vector databases in the [Weaviate developer docs](https://weaviate.io/developers/weaviate)

If running locally, to ensure smooth execution and prevent potential conflicts with your global Python environment, we recommend following the instructions in the README running the code in a virtual environment.

<a id='TOC'></a>  
## Table of contents  

1. <a href="#Dependencies">Dependencies</a><br>
2. <a href="#Connecting">Connecting to your Weaviate cluster</a><br>
3. <a href="#Collections">Setting up collections and loading data</a><br>
   3.1 <a href="#BrandsCollection">Create and populate Brands collection</a><br>
   3.2 <a href="#EcommerceCollection">Create and populate Ecommerce collection</a><br>
   3.3 <a href="#BooksCollection">Create and populate Books collection</a><br>
4. <a href="#QueryAgent">Creating the Query Agent</a><br>
5. <a href="#ExampleQueries">Example Query Agent interactions</a><br>
   5.1 <a href="#ClassicRAG">Classic RAG search</a><br>
   5.2 <a href="#DatabaseFiltering">Generated database filtering</a><br>
   5.3 <a href="#StatisticalQueries">Statistical and aggregation queries</a><br>
   5.4 <a href="#MultipleQueries">Querying multiple databases</a><br>
   5.5 <a href="#MissingInfo">Identifying missing information</a><br>

<a id='Dependencies'></a>  
## 1. Dependencies
[Back to table of contents](#TOC)

This section initializes the necessary dependencies and sets environment variables for connecting to your Weaviate Cloud instance.

**If using Colab, add your keys between the quotation marks in the cell below**

In [ ]:
# un this cell to load your keys in Colab
import os

# set environment variables
os.environ['WEAVIATE_URL'] = ''  # weaviate instance url
os.environ['WEAVIATE_KEY'] = ''  # admin api key

**If running locally, make sure your keys are in your .env file and run this cell**

In [2]:
# Run this cell if you are working locally in VS code or Code Space
import dotenv

dotenv.load_dotenv(override=True)

True

<a id='Connecting'></a>  
## 2. Connecting to your Weaviate cluster  
[Back to table of contents](#TOC)  

To interact with our Weaviate cluster, we'll initialize a client object and verify the connection is successful. This connection will be used throughout the notebook for all data operations and Query Agent interactions.

In [3]:
import weaviate, os

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.getenv("WEAVIATE_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WEAVIATE_KEY"))
)

# check if the connection is successful
client.is_ready()

True

<a id='Collections'></a>  
## 3. Setting up collections and loading data  
[Back to table of contents](#TOC)  

In this section, we'll create three collections (Brands, Ecommerce, and Books) and populate them with pre-vectorized data from the [Weaviate Agents dataset on Hugging Face](https://huggingface.co/datasets/weaviate/agents). Each collection uses the Snowflake Arctic Embed v2.0 model for consistent vector embeddings.

<a id='BrandsCollection'></a>  
### 3.1 Create and populate Brands collection  
[Back to table of contents](#TOC)  

We'll start by creating the Brands collection with properties for clothing brand information, then populate it with brand records from the Weaviate agents dataset.

In [10]:
# import required module for configuration
import weaviate.classes.config as wc

# delete collection if it exists
if client.collections.exists("Brands"):
    client.collections.delete("Brands")

client.collections.create(
    name="Brands", #Collection name

    #Set vectorizer configuration for text2vec using Weaviates embedding model
    vector_config=wc.Configure.Vectors.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        source_properties=["name", "description", "child_brands", "parent_brand"],
    ),

)

print("Successfully created collection: Brands.")

Successfully created collection: Brands.


In [11]:
# Upload brands data using the Weaviate-recommended streaming approach
from datasets import load_dataset

# Load fresh streaming dataset for upload
brands_dataset = load_dataset("weaviate/agents", "query-agent-brands", split="train", streaming=True)

brands_collection = client.collections.get("Brands")
with brands_collection.batch.fixed_size(batch_size=100) as batch:
    for item in brands_dataset:
        batch.add_object(
            properties=item["properties"],
            vector=item["vector"]
        )

print("Successfully uploaded brands data using Weaviate streaming method!")

Successfully uploaded brands data using Weaviate streaming method!


<a id='EcommerceCollection'></a>  
### 3.2 Create and populate Ecommerce collection  
[Back to table of contents](#TOC)  

Next, we'll create the Ecommerce collection for fashion items with properties including prices, categories, reviews, and brand associations, then load product records.

In [12]:
# import required module for configuration
import weaviate.classes.config as wc

# delete collection if it exists
if client.collections.exists("Ecommerce"):
    client.collections.delete("Ecommerce")

client.collections.create(
    name="Ecommerce", #Collection name

    #Set vectorizer configuration for text2vec using Weaviates embedding model
    vector_config=wc.Configure.Vectors.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        source_properties=["name", "description", "collection", "category", "subcategory", "brand", "colors", "tags", "reviews"],
    ),
)

print("Successfully created collection: Ecommerce.")

Successfully created collection: Ecommerce.


In [13]:
# Upload ecommerce data using the Weaviate-recommended streaming approach
ecommerce_dataset = load_dataset("weaviate/agents", "query-agent-ecommerce", split="train", streaming=True)

ecommerce_collection = client.collections.get("Ecommerce")
with ecommerce_collection.batch.fixed_size(batch_size=100) as batch:
    for item in ecommerce_dataset:
        batch.add_object(
            properties=item["properties"],
            vector=item["vector"]
        )

print("Successfully uploaded ecommerce data using Weaviate streaming method!")

Successfully uploaded ecommerce data using Weaviate streaming method!


<a id='BooksCollection'></a>  
### 3.3 Create and populate Books collection  
[Back to table of contents](#TOC)  

Finally, we'll create the Books collection with properties for titles, authors, descriptions, and genres, then populate it with book records from various genres including mystery, fiction, and non-fiction.

In [14]:
# import required module for configuration
import weaviate.classes.config as wc

# delete collection if it exists
if client.collections.exists("Books"):
    client.collections.delete("Books")

client.collections.create(
    name="Books", #Collection name

    #Set vectorizer configuration for text2vec using Weaviates embedding model
    vector_config=wc.Configure.Vectors.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        source_properties=["title", "author", "description", "genres"],
    ),
)

print("Successfully created collection: Books.")

Successfully created collection: Books.


In [15]:
# Upload books data using the Weaviate-recommended streaming approach
books_dataset = load_dataset("weaviate/agents", "query-agent-books", split="train", streaming=True)

books_collection = client.collections.get("Books")
with books_collection.batch.fixed_size(batch_size=100) as batch:
    for item in books_dataset:
        batch.add_object(
            properties=item["properties"],
            vector=item["vector"]
        )

print("Successfully uploaded books data using Weaviate streaming method!")

Successfully uploaded books data using Weaviate streaming method!


In [16]:
# import required module for configuration
import weaviate.classes.config as wc

# delete collection if it exists
if client.collections.exists("Financialcontracts"):
    client.collections.delete("Financialcontracts")

client.collections.create(
    name="Financialcontracts", #Collection name

    #Set vectorizer configuration for text2vec using Weaviates embedding model
    vector_config=wc.Configure.Vectors.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        source_properties=["contract_text"],
    ),
)

print("Successfully created collection: Financialcontracts.")

Successfully created collection: Financialcontracts.


In [17]:
# Upload books data using the Weaviate-recommended streaming approach
books_dataset = load_dataset("weaviate/agents", "query-agent-financial-contracts", split="train", streaming=True)

books_collection = client.collections.get("Financialcontracts")
with books_collection.batch.fixed_size(batch_size=100) as batch:
    for item in books_dataset:
        batch.add_object(
            properties=item["properties"],
            vector=item["vector"]
        )

print("Successfully uploaded Financialcontracts data using Weaviate streaming method!")

Successfully uploaded Financialcontracts data using Weaviate streaming method!


<a id='QueryAgent'></a>  
## 4. Creating the Query Agent  
[Back to table of contents](#TOC)  

Now we'll create our Query Agent instance, which will intelligently route queries across our three collections (Brands, Ecommerce, Books) and automatically determine the optimal search strategy.

The output from the Query Agent will include: the original query, the generated answer to the query, the searches performed, the collections searched, filters applied, aggregates completed, source objects pulled from the database that comntributed to the generated answer and any mising information if the generated answer is incomplete.

In [4]:
from weaviate.agents.query import QueryAgent

# Instantiate a new agent object, and specify the collections to query
qa = QueryAgent(
    client=client,
    collections=["Brands", "Ecommerce", "Books", "Financialcontracts"],
)


<a id='ExampleQueries'></a>  
## 5. Example Query Agent interactions  
[Back to table of contents](#TOC)  

Let's explore the Query Agent's capabilities through various types of queries. Notice how the agent automatically determines which collections to search, applies filters, and performs aggregations based on the natural language input.

<a id='BookQueries'></a>  
### 5.0 Classic Vector Retrieval 
[Back to table of contents](#TOC)  

This Query Agent is able to perform classic vector database retrieval without initiating a subsequent generative step.

In [5]:
# Perform a vector search
response = qa.search(
    "Books about plants"
)

# Print the response
# The output will include: the original query, the generated answer to the query, the seraches performed,
# the collections searched, filters applied, aggregates completed and source objects ulled from the database.
print(response)

searches=[QueryResultWithCollectionNormalized(query='books about plants or botany or gardening', filters=TextArrayPropertyFilter(property_name='genres', operator=<ComparisonOperator.CONTAINS_ANY: 'contains_any'>, value=['plants', 'botany', 'gardening']), collection='Books')] usage=ModelUnitUsage(model_units=117, usage_in_plan=True, remaining_plan_requests=834) total_time=4.347671747207642 search_results=QueryReturn(objects=[Object(uuid=UUID('36827096-8787-4e34-b6dd-3378bc4586ed'), metadata={'creation_time': None, 'last_update_time': None, 'distance': None, 'certainty': None, 'score': 0.77734375, 'explain_score': None, 'is_consistent': None, 'rerank_score': None}, properties={'title': "Botanical Shakespeare: An Illustrated Compendium of All the Flowers, Fruits, Herbs, Trees, Seeds, and Grasses Cited by the World's Greatest Playwright", 'description': 'A captivating, beautifully illustrated, one-of-a-kind color compendium of the flowers, fruits, herbs, trees, seeds, and grasses cited in 

<a id='BookQueries'></a>  
### 5.1 Classic RAG search  
[Back to table of contents](#TOC)  

This Query Agent is able to respond as a classic RAG system. The Query Agent will run an initial generative step that will optimize the original query for a vector database.



In [6]:
# Perform a query
response = qa.ask(
    "Are there any books about King Arthur or Knights that I should read?"
)

# Print the response
# The output will include: the original query, the generated answer to the query, the seraches performed,
# the collections searched, filters applied, aggregates completed and source objects ulled from the database.
response.display()

AskModeResponse(
│   output_type='final_state',
│   searches=[
│   │   QueryResultWithCollectionNormalized(
│   │   │   query='King Arthur',
│   │   │   filters=FilterAndOr(
│   │   │   │   combine='OR',
│   │   │   │   filters=[
│   │   │   │   │   TextPropertyFilter(
│   │   │   │   │   │   property_name='title',
│   │   │   │   │   │   operator=<ComparisonOperator.LIKE: 'LIKE'>,
│   │   │   │   │   │   value='*King Arthur*'
│   │   │   │   │   ),
│   │   │   │   │   TextPropertyFilter(
│   │   │   │   │   │   property_name='description',
│   │   │   │   │   │   operator=<ComparisonOperator.LIKE: 'LIKE'>,
│   │   │   │   │   │   value='*King Arthur*'
│   │   │   │   │   )
│   │   │   │   ]
│   │   │   ),
│   │   │   collection='Books'
│   │   ),
│   │   QueryResultWithCollectionNormalized(
│   │   │   query='knights',
│   │   │   filters=TextPropertyFilter(
│   │   │   │   property_name='description',
│   │   │   │   operator=<ComparisonOperator.LIKE: 'LIKE'>,
│   │   │   │   value='*knight*'
│   │   │   ),
│   │   │   collection='Books'
│   │   )
│   ],
│   aggregations=[],
│   usage=ModelUnitUsage(model_units=238, usage_in_plan=True, remaining_plan_requests=830),
│   total_time=6.422454833984375,
│   is_partial_answer=False,
│   missing_information=[],
│   final_answer='If you\'re interested in books about King Arthur, a great recommendation is "The Winter King" by Bernard Cornwell. It tells the story of King Arthur as a warlord trying to unite Britain amidst chaos and Saxon invasions, featuring elements of romance and magic.\n\nFor books focused on knights, here are some options:\n\n- "The Rogue Blades" by Dave Kish centers on knights and warriors involved in dueling contests within a kingdom ruled by a peace-loving queen.\n- "Dawn of the Knight I - Xeltian Invitation" by Steven M. Vincent is a fantasy tale about knights fighting dark forces to protect mankind.\n- "The Way of Kings" by Brandon Sanderson is an epic fantasy featuring the legendary Knights Radiant, ancient orders of mystical warriors, and their impact on a war-torn world.\n\nThese books cover a range of knightly themes, from Arthurian legend to fantasy wars and heroic duels.',
│   sources=[
│   │   Source(object_id='9536831d-6e02-4462-8af5-e630235a6582', collection='Books'),
│   │   Source(object_id='7a1ea585-48e4-465a-a85b-bc14bd710f85', collection='Books'),
│   │   Source(object_id='c8093ebc-3271-455f-a599-b02b91d44864', collection='Books'),
│   │   Source(object_id='ccc58d9b-b57a-435b-bb90-af9abfc37a25', collection='Books')
│   ]
)

<a id='BookQueries'></a>  
### 5.2 Generated database filtering   
[Back to table of contents](#TOC)  

This Query Agent is able determine appropriate filters and apply filtering for database objects returned by the vector database query. This allows for dynamic filtering in RAG systems, a step that is normally a manual process.


In [7]:
# Perform a query
response = qa.ask(
    "I am in the mood for a good mystery book, what should I read?"
)

# Print the response
response.display()

AskModeResponse(
│   output_type='final_state',
│   searches=[
│   │   QueryResultWithCollectionNormalized(
│   │   │   query=None,
│   │   │   filters=TextArrayPropertyFilter(
│   │   │   │   property_name='genres',
│   │   │   │   operator=<ComparisonOperator.CONTAINS_ANY: 'contains_any'>,
│   │   │   │   value=['mystery']
│   │   │   ),
│   │   │   collection='Books'
│   │   )
│   ],
│   aggregations=[],
│   usage=ModelUnitUsage(model_units=206, usage_in_plan=True, remaining_plan_requests=826),
│   total_time=8.714638471603394,
│   is_partial_answer=False,
│   missing_information=[],
│   final_answer='If you\'re in the mood for a good mystery book, here are several great options across different styles of mystery:\n\n1. "The Wife Between Us" by Greer Hendricks and Sarah Pekkanen - A psychological, twist-filled mystery about assumptions that will keep you guessing about a tangled love triangle.\n\n2. "Damascus Cover" by Howard Kaplan - A gripping espionage thriller with plenty of suspense and twists, involving a spy mission full of danger and intrigue.\n\n3. "Maisie Dobbs" by Jacqueline Winspear - A historical mystery featuring a psychologist and investigator uncovering a complex case tied to World War I.\n\n4. "The Dark Web Murders" by Brian O\'Hare - A thrilling mystery involving a cold-blooded, philosophical serial killer and a harrowing investigation.\n\n5. "Gone, Baby, Gone" by Dennis Lehane - A gritty, emotional mystery in a tough Boston neighborhood involving private investigators searching for a missing child.\n\n6. "Behind Closed Doors" by B.A. Paris - A psychological thriller exploring the dark secrets behind a seemingly perfect marriage.\n\n7. "The Black Book" by Orhan Pamuk - An unconventional and thought-provoking mystery set in Istanbul, blending literary questions with a search for a missing wife.\n\n8. "The Girl in the Spider\'s Web" by David Lagercrantz - A suspenseful and modern thriller featuring a hacker and journalist chasing dangerous secrets.\n\n9. "Live to Tell" by Lisa Gardner - A brutal family murder mystery investigated by a determined police detective.\n\nEach offers a compelling mystery with its own unique style and atmosphere, so you could pick based on what kind of mystery mood you are in—psychological, historical, thriller, or classic detective style. Enjoy your reading!',
│   sources=[
│   │   Source(object_id='00bf10f5-0fe9-42c8-a36f-df2cc92d5562', collection='Books'),
│   │   Source(object_id='108fc3de-54dc-4396-bbc8-cc042e15ca88', collection='Books'),
│   │   Source(object_id='1e81b838-a4f3-41cb-bffb-1d99385cb150', collection='Books'),
│   │   Source(object_id='22fa5bb7-e2ce-4f37-9e49-07e7301aee69', collection='Books'),
│   │   Source(object_id='23c1ba6f-717c-4b69-8e47-457c2245832e', collection='Books'),
│   │   Source(object_id='3a24eeae-aa51-4b1e-8784-9fb4bfcad6f0', collection='Books'),
│   │   Source(object_id='4f597f16-5e32-4f51-9333-509a70ca0e04', collection='Books'),
│   │   Source(object_id='6dcefd0a-3eb8-4e76-83d5-015e1ab672af', collection='Books'),
│   │   Source(object_id='452213a6-5d88-4d0d-a21c-98c1f2017635', collection='Books')
│   ]
)

<a id='StatisticalQueries'></a>  
### 5.3 Statistical and aggregation queries  
[Back to table of contents](#TOC)  

The Query Agent automatically recognizes when aggregation is needed and performs complex statistical operations like counting and grouping data.

In [8]:
# Perform a query
response = qa.ask(
    "Which author has the most books in my collection?"
)

# Print the response
response.display()

AskModeResponse(
│   output_type='final_state',
│   searches=[],
│   aggregations=[
│   │   AggregationResultWithCollectionNormalized(
│   │   │   groupby_property='author',
│   │   │   aggregation=TextPropertyAggregation(
│   │   │   │   property_name='title',
│   │   │   │   metrics=<TextMetrics.COUNT: 'COUNT'>,
│   │   │   │   top_occurrences_limit=None
│   │   │   ),
│   │   │   filters=None,
│   │   │   collection='Books'
│   │   )
│   ],
│   usage=ModelUnitUsage(model_units=560, usage_in_plan=True, remaining_plan_requests=822),
│   total_time=5.935815095901489,
│   is_partial_answer=False,
│   missing_information=[],
│   final_answer='The author with the most books in your collection is Stephen King, with a total of 57 books.',
│   sources=[]
)

<a id='MultipleQueries'></a>  
### 5.4 Querying multiple databases  
[Back to table of contents](#TOC)  

The Query agent looks at collection descriptions and property data to determine which collection or collections to query for proper context for the query response generation.

In [9]:
# Perform a query
response = qa.ask(
    """I am looking to buy a cool new jean jacket or light coat at or below $150.
    I want a trendy but also timeless look but can also hold up and last me a long time.
    If possible, I want to purchase something from a new company that might get me a
    sponsorship for my social media account."""
)
# Print the response
response.display()

AskModeResponse(
│   output_type='final_state',
│   searches=[
│   │   QueryResultWithCollectionNormalized(
│   │   │   query='jean jacket',
│   │   │   filters=IntegerPropertyFilter(
│   │   │   │   property_name='price',
│   │   │   │   operator=<ComparisonOperator.LESS_EQUAL: '<='>,
│   │   │   │   value=150.0
│   │   │   ),
│   │   │   collection='Ecommerce'
│   │   ),
│   │   QueryResultWithCollectionNormalized(
│   │   │   query='light coat',
│   │   │   filters=FilterAndOr(
│   │   │   │   combine='AND',
│   │   │   │   filters=[
│   │   │   │   │   IntegerPropertyFilter(
│   │   │   │   │   │   property_name='price',
│   │   │   │   │   │   operator=<ComparisonOperator.LESS_EQUAL: '<='>,
│   │   │   │   │   │   value=150.0
│   │   │   │   │   ),
│   │   │   │   │   TextPropertyFilter(
│   │   │   │   │   │   property_name='subcategory',
│   │   │   │   │   │   operator=<ComparisonOperator.LIKE: 'LIKE'>,
│   │   │   │   │   │   value='*coat*'
│   │   │   │   │   )
│   │   │   │   ]
│   │   │   ),
│   │   │   collection='Ecommerce'
│   │   ),
│   │   QueryResultWithCollectionNormalized(
│   │   │   query='fashion brands sponsorship opportunities',
│   │   │   filters=IntegerPropertyFilter(
│   │   │   │   property_name='foundation_year',
│   │   │   │   operator=<ComparisonOperator.GREATER_EQUAL: '>='>,
│   │   │   │   value=2020.0
│   │   │   ),
│   │   │   collection='Brands'
│   │   )
│   ],
│   aggregations=[
│   │   AggregationResultWithCollectionNormalized(
│   │   │   groupby_property=None,
│   │   │   aggregation=IntegerPropertyAggregation(
│   │   │   │   property_name='price',
│   │   │   │   metrics=<NumericMetrics.COUNT: 'COUNT'>
│   │   │   ),
│   │   │   filters=FilterAndOr(
│   │   │   │   combine='AND',
│   │   │   │   filters=[
│   │   │   │   │   TextPropertyFilter(
│   │   │   │   │   │   property_name='name',
│   │   │   │   │   │   operator=<ComparisonOperator.LIKE: 'LIKE'>,
│   │   │   │   │   │   value='*jean jacket*'
│   │   │   │   │   ),
│   │   │   │   │   IntegerPropertyFilter(
│   │   │   │   │   │   property_name='price',
│   │   │   │   │   │   operator=<ComparisonOperator.LESS_EQUAL: '<='>,
│   │   │   │   │   │   value=150.0
│   │   │   │   │   )
│   │   │   │   ]
│   │   │   ),
│   │   │   collection='Ecommerce'
│   │   ),
│   │   AggregationResultWithCollectionNormalized(
│   │   │   groupby_property=None,
│   │   │   aggregation=TextPropertyAggregation(
│   │   │   │   property_name='name',
│   │   │   │   metrics=<TextMetrics.COUNT: 'COUNT'>,
│   │   │   │   top_occurrences_limit=None
│   │   │   ),
│   │   │   filters=FilterAndOr(
│   │   │   │   combine='AND',
│   │   │   │   filters=[
│   │   │   │   │   TextPropertyFilter(
│   │   │   │   │   │   property_name='subcategory',
│   │   │   │   │   │   operator=<ComparisonOperator.LIKE: 'LIKE'>,
│   │   │   │   │   │   value='*coat*'
│   │   │   │   │   ),
│   │   │   │   │   IntegerPropertyFilter(
│   │   │   │   │   │   property_name='price',
│   │   │   │   │   │   operator=<ComparisonOperator.LESS_EQUAL: '<='>,
│   │   │   │   │   │   value=150.0
│   │   │   │   │   ),
│   │   │   │   │   TextPropertyFilter(
│   │   │   │   │   │   property_name='colors',
│   │   │   │   │   │   operator=<ComparisonOperator.LIKE: 'LIKE'>,
│   │   │   │   │   │   value='*light*'
│   │   │   │   │   )
│   │   │   │   ]
│   │   │   ),
│   │   │   collection='Ecommerce'
│   │   )
│   ],
│   usage=ModelUnitUsage(model_units=576, usage_in_plan=True, remaining_plan_requests=818),
│   total_time=13.767515420913696,
│   is_partial_answer=False,
│   missing_information=[],
│   final_answer='For a trendy yet timeless jean jacket or light coat priced at or below $150, here are some stylish options from relatively new and quality-conscious brands:\n\n1. Garden Dusk Linen Jacket by Loom & Aura ($90)  \n- A soft linen jacket in a subtle lavender color, tailored for comfort with thoughtful detailing.  \n- It brings both charm and durability, suitable for various 

<a id='MissingInfo'></a>  
### 5.5 Identifying missing information  
[Back to table of contents](#TOC)  

If a partial response is generated for a query the Query Agent will report the missing information not present in the database.

In [10]:
# Perform a query
response = qa.ask(
    """I want to buy a new leather jacket but I am vegan and only buy from socially responsible brands."""
)
# Print the response
response.display()

AskModeResponse(
│   output_type='final_state',
│   searches=[
│   │   QueryResultWithCollectionNormalized(
│   │   │   query='vegan leather jacket socially responsible brand ethical sustainable cruelty-free',
│   │   │   filters=TextPropertyFilter(
│   │   │   │   property_name='subcategory',
│   │   │   │   operator=<ComparisonOperator.LIKE: 'LIKE'>,
│   │   │   │   value='*jacket*'
│   │   │   ),
│   │   │   collection='Ecommerce'
│   │   ),
│   │   QueryResultWithCollectionNormalized(
│   │   │   query='socially responsible brands ethical brands sustainable brands vegan brands',
│   │   │   filters=None,
│   │   │   collection='Brands'
│   │   )
│   ],
│   aggregations=[],
│   usage=ModelUnitUsage(model_units=252, usage_in_plan=True, remaining_plan_requests=814),
│   total_time=10.90539264678955,
│   is_partial_answer=True,
│   missing_information=[
│   │   'Specific confirmation that the brands Vivid Verse and Canvas & Co. are vegan or socially responsible is missing, as this was not found in the sources.',
│   │   'Detailed information or certifications about the social responsibility, ethical practices, or vegan credentials of the mentioned brands (Vivid Verse, Canvas & Co., Nova Nest) is not provided in the available results.'
│   ],
│   final_answer='If you\'re vegan and want to buy a new leather jacket from socially responsible and ethical brands, looking for vegan leather or faux leather alternatives is key. Among the available options, the "Shroud Jacket" by Vivid Verse could be a great pick, as it uses ultra-resistant fabric and is not traditional leather. However, the brand specifics about its social responsibility or vegan certification are not explicitly mentioned.\n\nBoth Canvas & Co. jackets ("Rustic Wanderer Utility Jacket" and "Rustic Patcher Jacket") use durable, weathered fabrics rather than leather, which might align better with vegan preferences. These brands emphasize durability and outdoor utility but do not state specific socially responsible credentials in the information given.\n\nFor truly vegan and socially responsible fashion, keep an eye on brands like Nova Nest and its associated brands (including Vivid Venture), which have a reputation for innovative design and contemporary style. Although detailed ethical or vegan-focused credentials are not listed here, these brands could be worth exploring further for sustainable fashion options.\n\nIn summary, your best options here are jackets made from vegan-friendly materials such as those from Vivid Verse and Canvas & Co., but you\'ll want to verify each brand\'s social responsibility policies directly, as that information isn\'t provided in detail.',
│   sources=[
│   │   Source(object_id='ea0bfa17-94fc-4af4-ab90-8ac34a4efb31', collection='Ecommerce'),
│   │   Source(object_id='6325f602-b335-45d7-9714-7abefe7974ec', collection='Ecommerce'),
│   │   Source(object_id='09994007-e02c-4633-87ee-5bf329280c6a', collection='Ecommerce'),
│   │   Source(object_id='b120b799-95a1-4616-8268-265252224343', collection='Brands'),
│   │   Source(object_id='0a68452b-143f-4881-b025-df8ad8ebe899', collection='Brands')
│   ]
)

In [11]:
client.close()